# Boilerplate

There's probably a better way than what follows

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

What can we run?

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['EndpointW',
 'PacketSettingsW',
 'PacketW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'TransportCmdW',
 'TransportInterface',
 'new_transport_interface']

## Transport Setup

Create a new transport interface, asynchronously. This uses the Python `asyncio` event loop, which a Jupyter Notebook already has running.

In [3]:
tiface_fut = new_transport_interface("0.0.0.0", 12346)
tiface_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f893132f750>()]>

Await the future to get the `TransportInterface`. Note: unlike in Rust, `await` isn't necessary to drive Future completion.

In [4]:
tiface = await tiface_fut
tiface

TransportInterface{ transport: Some(<Transport>),   cmd_tx: Sender { chan: Tx { inner: Chan { tx: Tx { block_tail: 0x7f89040133e0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   response_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7f8904013fd0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   notify_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7f89040143c0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } } }

### Run the Transport in background
Here is the fun part :)

In [5]:
run_fut = tiface.run()
run_fut

<Future pending cb=[<builtins.PyDoneCallback object at 0x7f893132f630>()]>

If the Future completed right away, it means something's fishy.

In [6]:
assert not run_fut.done()

Note the `transport: None` at the beginning of the following `repr` output. It is `None` because `run()` takes the `Transport`.

In [7]:
tiface

TransportInterface{ transport: None,   cmd_tx: Sender { chan: Tx { inner: Chan { tx: Tx { block_tail: 0x7f89040133e0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   response_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7f8904013fd0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } },   notify_rx: Receiver { chan: Rx { inner: Chan { tx: Tx { block_tail: 0x7f89040143c0, tail_position: 0 }, semaphore: (Semaphore { permits: 1000 }, 1000), rx_waker: AtomicWaker, tx_count: 1, rx_fields: "..." } } } }

## Transport Operation

Create a `ProcessUniqueId` wrapper.

In [8]:
puid = ProcessUniqueIdW()
puid

puid-0-0

Create another, and compare against the first.

In [9]:
puid2 = ProcessUniqueIdW()
print(repr(puid2))
assert puid != puid2

puid-0-1


Create a `PacketSettings` wrapper. Note that the unique ID `tid` is optional, and duration is in milliseconds.

In [10]:
packet_settings = PacketSettingsW(10)
packet_settings

PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 2 }, retry_interval: 10ms }

Make a transport command

In [11]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
chococat_endpoint

Endpoint(157.230.134.224:2016)

In [12]:
new_endpoint_transport_cmd = TransportCmdW("newendpoint", endpoint=chococat_endpoint, timeout=100)
new_endpoint_transport_cmd

NewEndpoint { endpoint: Endpoint(157.230.134.224:2016), timeout: 100ms }